# Base program ocr

In [1]:
import tensorflow as tf
import tensorflow.keras as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M 
import tensorflow.keras.optimizers as O
import tensorflow.keras.preprocessing.image as I

In [2]:
def ConvBlock(model, filters, kernel):
    model = L.Conv2D(filters, kernel-2, padding="same")(model)
    model = L.Conv2D(filters, kernel, padding = "same")(model)
    model = L.Conv2D(filters, kernel-2, padding="same")(model)
    model = L.BatchNormalization()(model)
    model = L.LeakyReLU(alpha=0.1)(model)
    model = L.MaxPool2D()(model)
    
    return model

In [3]:
def CNN(inp):
    inp = L.Input(inp)
    x = ConvBlock(inp, 32, 3)
    x = ConvBlock(x, 64, 3)
    x = L.Flatten()(x)
    x = L.Dense(100, activation="relu")(x)
    x = L.Dense(36, activation="softmax")(x)

    model = M.Model(inputs=inp, outputs=x, name="CNN")
    
    return model

In [4]:
model = CNN((32,32,3))

In [5]:
model.summary()

Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        128       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        1056      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 32)        0         
                                                               

# Data

In [6]:
TRAIN = r"C:\Users\suyash\Desktop\trainingData"
clas  = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
         'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
traindata = I.ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
            .flow_from_directory(directory=TRAIN, target_size=(32,32), classes=clas,batch_size=100)

Found 20628 images belonging to 36 classes.


In [7]:
model.compile(optimizer=O.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(traindata, epochs=5, verbose=1)

Epoch 1/5
207/207 [==============================] - 57s 273ms/step - loss: 0.4547 - accuracy: 0.8990
Epoch 2/5
207/207 [==============================] - 50s 242ms/step - loss: 0.3011 - accuracy: 0.9288
Epoch 3/5
207/207 [==============================] - 48s 231ms/step - loss: 0.2319 - accuracy: 0.9427
Epoch 4/5
207/207 [==============================] - 50s 241ms/step - loss: 0.1836 - accuracy: 0.9527
Epoch 5/5
207/207 [==============================] - 49s 238ms/step - loss: 0.1554 - accuracy: 0.9592


In [12]:
import numpy as np
def preprocess(file):
    # loc = r'C:/Users/suyash/Desktop\asl_alphabet_test\asl_alphabet_test'
    img = I.load_img(file, target_size=(32,32))
    img_array = I.img_to_array(img)
    img_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.vgg16.preprocess_input(img_dims)

In [13]:
link = r"C:\Users\suyash\Desktop\trainingData\3\3.png.png"
img = preprocess(link)
max(max(model.predict(img.reshape(-1,32,32,3))))

0.9989341

In [14]:
model.save("sasa.h5")